In [1]:
import folium
from folium import plugins
from folium.plugins import Fullscreen
from folium.plugins import MarkerCluster 

import requests
import xml.etree.ElementTree as ET

import base64
from BaseXClient import BaseXClient

import webbrowser

from tkinter import Tk
from tkhtmlview import HTMLLabel
from tkinter.filedialog import askopenfilename
from PIL import ImageTk, Image

import PyQt5
from PyQt5 import *
from PyQt5.QtWidgets import *
from PyQt5.QtCore import *
from PyQt5.QtWebEngineWidgets import *
from PyQt5.QtGui import *

import time
import threading
import datetime
from reportlab.pdfgen import canvas

import sys
import os
import re
import random

In [2]:
def cargaDBdadoUnMapa(session, mapGiven):
    session.execute("create db osm")
    try:
        with open(mapGiven, 'rb') as f:
            contenido = f.read()
            session.add("map.osm", contenido.decode())
        # Imprimir un mensaje de confirmación
        print("Mapa OSM cargado en BaseX")
    except Exception as e:
          # Imprimir un mensaje de error si hay un problema
          print("Error al cargar el mapa OSM en BaseX:", e)

def cargaDB2(session):
    session.execute("create db osm")
    arch = ['MT-OSM-master/TAGS/datasets/Madrid/map.osm','MT-OSM-master/TAGS/datasets/SaoPaolo/map.osm',
           'MT-OSM-master/TAGS/datasets/London/map.osm']
    nrandom = random.randint(0, 2)
    
    try:
        with open(arch[nrandom], 'rb') as f:
            contenido = f.read()
            session.add("map.osm", contenido.decode())
            query = session.query("let $doc := doc('osm') return string(($doc//way)[1]/tag[@k='name']/@v)")
            result = query.execute()

    # Obtener el resultado como una cadena de texto
            nombreCalle = result.strip()
    
            
        # Imprimir un mensaje de confirmación
        print("Mapa OSM cargado en BaseX")
        return nombreCalle
    
    except Exception as e:
          # Imprimir un mensaje de error si hay un problema
          print("Error al cargar el mapa OSM en BaseX:", e)
        
def cargaDBradio(lat, lon, radio, session):
    consulta = f"""
    [out:xml];
    (
        node(around:{radio},{lat},{lon});
        way(around:{radio},{lat},{lon});
        relation(around:{radio},{lat},{lon});
    );
    out meta;
    """

    # URL de la API de Overpass
    url = "https://overpass-api.de/api/interpreter"

    # Parámetros de la solicitud POST
    parametros = {
        "data": consulta
    }

    # Realizar la solicitud POST a Overpass API
    response = requests.post(url, data=parametros)

    # Verificar el código de estado de la respuesta
    if response.status_code == 200:
        # Decodificar la respuesta en formato XML
        osm_data = ET.fromstring(response.content)

        try:
            # Crear una base de datos en BaseX
            session.execute("create db osm")

            # Convertir el objeto Element a una cadena
            osm_data_str = ET.tostring(osm_data, encoding="unicode", method="xml")

            # Añadir el contenido XML a la base de datos
            session.add("map.osm", osm_data_str)

            # Imprimir un mensaje de confirmación
            print("Mapa OSM cargado en BaseX")
        except Exception as e:
            # Imprimir un mensaje de error si hay un problema
            print("Error al cargar los datos OSM en BaseX:", e)
    else:
        print("Error al descargar los datos OSM desde Overpass API")
        
def cargaDBcalle(nombre_calle, session):
    # Consulta Overpass para obtener los nodos de la calle por nombre
    consulta = f"""
    [out:xml];
    way["name"="{nombre_calle}"];
    (._;>;);
    out;
    """

    # URL de la API de Overpass
    url = "https://overpass-api.de/api/interpreter"

    # Parámetros de la solicitud POST
    parametros = {
        "data": consulta
    }

    # Realizar la solicitud POST a Overpass API
    response = requests.post(url, data=parametros)

    # Verificar el código de estado de la respuesta
    if response.status_code == 200:
        # Decodificar la respuesta en formato XML
        osm_data = ET.fromstring(response.content)

        try:
            # Crear una base de datos en BaseX
            session.execute("create db osm")

            # Convertir el objeto Element a una cadena
            osm_data_str = ET.tostring(osm_data, encoding="unicode", method="xml")

            # Añadir el contenido XML a la base de datos
            session.add("map.osm", osm_data_str)

            # Imprimir un mensaje de confirmación
            print(f"Nodos de la calle '{nombre_calle}' cargados en BaseX")
        except Exception as e:
            # Imprimir un mensaje de error si hay un problema
            print(f"Error al cargar los nodos de la calle '{nombre_calle}' en BaseX:", e)
    else:
        print(f"Error al descargar los nodos de la calle '{nombre_calle}' desde Overpass API")

        
def meterMapa(session):
    # Seleccionamos el archivo
    raiz = Tk()
    raiz.withdraw()
    mapGiven = askopenfilename()
    
    # Cargamos el mapa dado el mapa seleccionado
    cargaDBdadoUnMapa(session, mapGiven)
    # Ejecutar la consulta XQuery para obtener el valor del atributo "ref" del primer "nd"
    query = session.query("let $doc := doc('osm') return string(($doc//way)[1]/tag[@k='name']/@v)")
    result = query.execute()

    # Obtener el resultado como una cadena de texto
    nombreCalle = result.strip()
    
    return nombreCalle

# Coloca el mapa según la dirección dada
def posicionarMapaConCalle(direccion):
    # Convertimos la dirección a coordenadas utilizando la API de OSM
    respuesta = requests.get("https://nominatim.openstreetmap.org/search",
                         params={"q": direccion, "format": "json"})
    # Tomamos la primera coincidencia
    resultado = respuesta.json()[0]
    # Metemos las coordenadas en el mapa
    generar_mapa(resultado["lat"], resultado["lon"], 16)
    
    return resultado["lat"], resultado["lon"]

# Busca los errores en el mapa
def MtGeometry(session, func, nombreCalle):
    r = []
    module_namespace = 'mt'
    module_path = 'MT-OSM-master\GEOMETRY\src\mt-osm-geometry.xqm'
    inp = f"import module namespace mt = '{module_namespace}' at '{module_path}';"
    
    inp += 'mt:' + func + '(.,"' + nombreCalle + '",())'
    result = session.query(inp)
    r += result.execute()

    return r

# Dado un id de NODO calcula su latitud y longitud
def latandlon(id):
    url = f"https://api.openstreetmap.org/api/0.6/node/{id}"
    response = requests.get(url)
    root = ET.fromstring(response.content)
    node = root.find("node")
    lat = node.attrib["lat"]
    lon = node.attrib["lon"]

    return lat, lon

# Marca los puntos del resultado de la consulta
def marcaPuntos(result, tipo, i, mapa):
    # Generamos un nuevo mapa ya que no tenemos acceso al que se muestra
    colors = ['orange', 'green', 'blue', 'red', 'black', 'cadetblue', 'gray', 'purple', 'pink']
   
    errores = "- " + tipo + "\n"
    nombre_calle = ""
    #Compacta el string en un xml
    xml_string = '<errores>'
    for r in result:
        xml_string += r
    xml_string+= '</errores>'
    
    
    root = ET.fromstring(xml_string)
    
    #Busca todas las ways
    ways = root.findall('.//way')
    
    #Por cada way, encuentra el primer nodo y lo marca en el mapa
    for w in ways:
        nd_element = w.find('./nd').get('ref')
        lat, lon= latandlon(nd_element)
        errores += "Nodo: " + nd_element 
        for etiqueta in w.findall(".//tag"):
            if etiqueta.get("k") == "name":
                nombre_calle = etiqueta.get("v")
        errores += " | " + nombre_calle + "\n"
        
        #Marca un punto en el mapa dado una [latitud, longitud]
        folium.Marker(location=[lat, lon], icon=folium.Icon(color=colors[i]), popup=tipo).add_to(mapa)
         
    
    xmlS = "- " + tipo + "\n"
    xmlS += xml_string
    
    return xmlS, errores, mapa

# Dibuja en m el área que definen las coordenadas
def dibujarArea(coordenadas, m):
    i = 1
    for i in range(len(coordenadas)):
        if i == len(coordenadas) - 1:
            folium.PolyLine(locations=[coordenadas[i], coordenadas[0]]).add_to(m)
        folium.PolyLine(locations=[coordenadas[i], coordenadas[i-1]]).add_to(m)
        i+=1
        
session = ""

In [3]:
# Define el tamaño de la ventana
def dimension():
    pantalla = QApplication.desktop()
    pantallaD = pantalla.screenGeometry(pantalla.primaryScreen())
    anchoP = int(pantallaD.width() * 0.75)
    alturaP = int(pantallaD.height() * 0.75)
    anchoV = int((pantallaD.width() - anchoP) / 2)
    alturaV = int((pantallaD.height() - alturaP) / 2)

    return anchoP, alturaP, anchoV, alturaV
    
def generar_mapa(lat, lon, zoom):
    mapa = folium.Map(location=[lat, lon], zoom_start=zoom)
    
    # Añadimos las opciones de selección de área
    draw_options = {'polygon': True,
                    'polyline': False,
                    'rectangle': True,
                    'circle': True,
                    'marker': False,
                    'circlemarker': False}
    draw_button = plugins.Draw(export=False, draw_options=draw_options)
    draw_button.add_to(mapa)
    
    mapa.save('mapa.html')
    
def mensaje_Error():
    mensaje = QMessageBox()
    mensaje.setWindowTitle("Mensaje")
    mensaje.setText("No se ha seleccionado ningún filtro, por favor seleecione alguno antes de continuar")
    mensaje.exec_()

strErrores = ""
strXml = ""
    
def filtros_seleccionados(checkboxes, nombresT):
    global strErrores
    global strXml
    global session
    
    strErrores = ""
    strXml = ""
    it = 0
    mapa = folium.Map(location=[40.4168, -3.7038], zoom_start=13)
    
    filtros_seleccionados = []
    for i, checkbox in enumerate(checkboxes):
        if checkbox.isChecked():
            filtros_seleccionados.append(nombresT[i])
    if len(filtros_seleccionados)==0:
        mensaje_Error();
    else:
        # Ejecutar la consulta XQuery para obtener el valor del atributo "ref" del primer "nd"
        query = session.query("let $doc := doc('osm') return string(($doc//way)[1]/tag[@k='name']/@v)")
        result = query.execute()

        # Obtener el resultado como una cadena de texto
        nombreCalle = result.strip()
        
        for fil in filtros_seleccionados:
            
            r = MtGeometry(session, fil, nombreCalle)
            xml, errores, mapa = marcaPuntos(r, fil, it, mapa)
            strErrores += errores
            strXml += xml + "\n"
            strXml += "----------------------------" + "\n"
            strErrores += "----------------------------" + "\n"
            it+=1
            
        # Guardamos el nuevo mapa marcado para mostrarlo
        mapa.fit_bounds(mapa.get_bounds())
        mapa.save('mapaR.html')
    
def zona_busqueda(lista2):
        botones_seleccionados = lista2.checkedButton()
        if botones_seleccionados:
            print("Botón seleccionado:", botones_seleccionados.text())
        else:
            print("Ningún botón seleccionado")

In [4]:
class AvanceBarra(threading.Thread):
    contador = 0
    
    def __init__(self, inicio):
        super().__init__()
        self.inicio = inicio
        self.contador = 0
    
    # Sobreescribimos la función para que haga lo que nosotros queramos
    def run(self):
        while self.contador <= 100:
            time.sleep(0.1)
            self.inicio.progreso.setValue(self.contador)
            self.contador += 2

class BotonRedondeado(QPushButton):
    def __init__(self, text, ancho, altura):
        super().__init__()
        
        # Añadimos propiedades del botón
        self.setCursor(Qt.PointingHandCursor)
        self.setFixedSize(ancho, altura)
        self.setStyleSheet(
            "*{background-color: white;" +
            "border-radius: 50px;" +
            "border: none;}" +
            "*:pressed {background-color: #C5DFF1;}" +
            "*:hover {background-color: #C5DFF1;}"
        )
        
        # Creamos un nuevo Layout para poder ajustar el texto
        boton = QHBoxLayout()
        texto = QLabel()

        texto.setText(text)
        texto.setStyleSheet(
            "*{color: #708dc7;" +
            "font: Gilroy;" +
            "font-size: 15pt;" +
            "font-weight: bold;" +
            "border: none;}"
        )
        texto.setAlignment(Qt.AlignCenter)
        texto.setWordWrap(True)
        
        # Deshabilitamos interacciones del Label para utilizar las del botón
        texto.setTextInteractionFlags(Qt.NoTextInteraction)
        texto.setMouseTracking(False)
        texto.setSizePolicy(QSizePolicy.Expanding, QSizePolicy.Expanding)
        
        # Añadimos el Label al Layout del botón
        boton.addWidget(texto)
        
        # Introducimos el Layout dentro del botón
        self.setText("")
        self.setLayout(boton)
        
class BotonRedondeadoMini(QPushButton):
    def __init__(self, text, ancho, altura):
        super().__init__()
        
        # Añadimos propiedades del botón
        self.setCursor(Qt.PointingHandCursor)
        self.setFixedSize(ancho, altura)
        self.setStyleSheet(
            "*{background-color: white;" +
            "border-radius: 20px;" +
            "border: none;}" +
            "*:pressed {background-color: #C5DFF1;}" +
            "*:hover {background-color: #C5DFF1;}"+ 
            "*:disabled {background-color: #CACCD3;" +
            "border: 2px grey;" +
            "border-style: inset;}"
        )
        
        # Creamos un nuevo Layout para poder ajustar el texto
        boton = QHBoxLayout()
        texto = QLabel()

        texto.setText(text)
        texto.setStyleSheet(
            "*{color: #708dc7;" +
            "font: Gilroy;" +
            "font-size: 10pt;" +
            "border: none;}"
        )
        texto.setAlignment(Qt.AlignCenter)
        texto.setWordWrap(True)
        
        # Deshabilitamos interacciones del Label para utilizar las del botón
        texto.setTextInteractionFlags(Qt.NoTextInteraction)
        texto.setMouseTracking(False)
        texto.setSizePolicy(QSizePolicy.Expanding, QSizePolicy.Expanding)
        
        # Añadimos el Label al Layout del botón
        boton.addWidget(texto)
        
        # Introducimos el Layout dentro del botón
        self.setText("")
        self.setLayout(boton)

class Inicio(QWidget):
    def __init__(self, anchoV, alturaV, anchoP, alturaP):
        super().__init__()
        
        # Creamos la Ventana Inicio
        self.resize(anchoV, alturaV)

        # Definimos el fondo
        self.fondo = QLabel(self)
        pixmap = QPixmap('img/fondo-grande.jpg')
        self.fondo.setPixmap(pixmap)
        self.fondo.setScaledContents(True)
        self.fondo.setGeometry(0, 0, int(anchoP*1.5), int(alturaP*1.5))
        
        # Creamos un Label central      
        self.frente = QLabel(self)
        self.frente.setGeometry(int(anchoP/4), int(alturaP/4 - 50), int(anchoP/2), int(alturaP/2))
        self.frente.setStyleSheet(
            "*{background-color: white;" +
            "color: #708dc7;" +
            "border-radius: 20px;" +
            "font-family: Gilroy;" +
            "font-size:25px;}"
        )
        self.frente.setAlignment(Qt.AlignCenter)  # Alineación del contenido
        self.frente.setText("<h1>¡BIENVENIDO!</h1><br><img src='img/logo-verde.png' width='280'>")
        self.frente.setContentsMargins(15, 15, 10, 10)
        
        # Creamos una barra de progreso para que parezca que el programa carga
        self.progreso = QProgressBar(self)
        self.progreso.setGeometry(int(anchoP/4 + 100), int(alturaP/4 - 50 + alturaP/2 + 50), int(anchoP/2 - 200), 40)
        self.progreso.setStyleSheet(
            "*{background-color: #708dc7;" +
            "border-radius: 5px;" +
            "border: 2px solid white;}" +
            "*:chunk {background-color: white;" +
            "margin: 1px;" +
            "width: 10px;}"
        )
        self.progreso.setTextVisible(False)
        self.progreso.setRange(0, 100)

        # Cambiamos ventana
        QTimer.singleShot(6000, self.ver_menu)
        
    def ver_menu(self):
        menu = Menu()
        self.parent().setCentralWidget(menu)
        self.close()

class Menu(QWidget):
    def __init__(self):
        super().__init__()
        
        # Definimos el fondo
        self.fondo = QLabel(self)
        pixmap = QPixmap('img/fondo-grande.jpg')
        self.fondo.setPixmap(pixmap)
        self.fondo.setScaledContents(True)
        self.fondo.setGeometry(0, 0, int(anchoP*1.5), int(alturaP*1.5))
        
        # Creamos el texto de arriba
        texto = QLabel()
        texto.setText("¿CÓMO DESEA REALIZAR LA CONSULTA?")
        texto.setStyleSheet(
            "*{color: white;" +
            "font: Gilroy;" +
            "font-size: 20pt;" +
            "font-weight: bold;}"
        )
        texto.setAlignment(Qt.AlignCenter)
        texto.setWordWrap(True)
        texto.setSizePolicy(QSizePolicy.Expanding, QSizePolicy.Expanding)

        # Creamos los botones de abajo
        botonesL = QHBoxLayout()
        b1 = BotonRedondeado("UTILIZAR POSICIÓN POR DEFECTO", 300, 200)
        b2 = BotonRedondeado("INTRODUCIR POSICIÓN", 300, 200)
        b3 = BotonRedondeado("CARGAR MAPA", 300, 200)
        
        b1.clicked.connect(self.ver_mapaD)
        b2.clicked.connect(self.pedir_posi)
        b3.clicked.connect(self.cargar_mapa)
        
        botonesL.addWidget(b1)
        botonesL.addWidget(b2)
        botonesL.addWidget(b3)
        
        botones = QLabel()
        botones.setLayout(botonesL)
        
        # Creamos un Label para introducir el logo en la esquina inferior
        icono = QLabel();
        icono.setPixmap(QPixmap("img/logo2.png"))
        icono.setAlignment(Qt.AlignRight)
        icono.setSizePolicy(QSizePolicy.Expanding, QSizePolicy.Fixed)
        
        # Creamos un Layout que contenga el texto y los botones
        contenedor = QVBoxLayout()
        contenedor.addStretch(0)
        contenedor.addStretch(0)
        contenedor.addWidget(texto)
        contenedor.addStretch(0)
        contenedor.addStretch(0)
        contenedor.addWidget(botones)
        contenedor.addStretch(0)
        contenedor.addWidget(icono)
        
        # Introducimos el Layout dentro del Widget
        self.setLayout(contenedor)
        
    def ver_mapaD(self):
        global session
        session = BaseXClient.Session('localhost', 1984, 'admin', 'admin')
        nombre = cargaDB2(session)
        mapaD = MapaD(posicionarMapaConCalle(nombre))
        self.parent().setCentralWidget(mapaD)
        self.close()
        
    def pedir_posi(self):
        pedirP = PedirPos(self.parent())
        
    def cargar_mapa(self):
        global session
        session = BaseXClient.Session('localhost', 1984, 'admin', 'admin')
        nombre = meterMapa(session)
        mapaD = MapaD(posicionarMapaConCalle(nombre))
        self.parent().setCentralWidget(mapaD)
        self.close()
        
class PedirPos(QDialog):
    def __init__(self, ventanaP):
        super().__init__()

        self.setWindowTitle("Solicitud de Posición")
        self.setWindowIcon(QIcon('img/logo2.ico'))
        self.setGeometry(int(anchoP/2), int(alturaP/2), 400, 270)
        
        # Imagen de fondo
        self.fondo = QLabel(self)
        pixmap = QPixmap('img/fondo-grande.jpg')
        self.fondo.setPixmap(pixmap)
        self.fondo.setScaledContents(True)
        self.fondo.setGeometry(0, 0, 400, 270)
        
        # Texto explicativo
        texto = QLabel("Introduzca la dirección destino:")
        texto.setStyleSheet(
            "*{font-family: 'Gilroy';" +
            "font-size: 23px;" +
            "color: white;}"
        )
        texto.setAlignment(Qt.AlignLeft)
        texto.setWordWrap(True)
        texto.setSizePolicy(QSizePolicy.Expanding, QSizePolicy.Expanding)
        
        # Cuadro para recoger la dirección
        direccion = QTextEdit()
        direccion.setStyleSheet(
            "*{font-family: 'Gilroy';" +
            "font-size: 21px;}" +
            "*:focus {border: 10px solid black;}"
        )
        direccion.setPlaceholderText("C/Av Nombre [Número], Población")
        direccion.setMaximumHeight(100)
        
        # Botones
        botonBox = QDialogButtonBox()
        botonBox.addButton(QDialogButtonBox.Ok)
        botonBox.addButton(QDialogButtonBox.Cancel)
        
        # Diseño del primero
        botonBox.button(QDialogButtonBox.Ok).setText("Continuar")
        botonBox.button(QDialogButtonBox.Ok).setStyleSheet(
            "*{background-color: white;" +
            "color: #708dc7;" +
            "font: Gilroy;" +
            "font-size: 20px;}" +
            "*:pressed {background-color:#C5DFF1;}" +
            "*:hover {background-color: #C5DFF1;}"
        )
        botonBox.button(QDialogButtonBox.Ok).setCursor(Qt.PointingHandCursor)
        try:
            botonBox.button(QDialogButtonBox.Ok).clicked.connect(lambda: self.enviar_direccion(direccion.toPlainText()))
        except Exception as e:
            print("Porfavor introduzca dirección")            
        
        # Diseño del segundo
        botonBox.button(QDialogButtonBox.Cancel).setText("Cancelar")
        botonBox.button(QDialogButtonBox.Cancel).setStyleSheet(
            "*{background-color: white;" +
            "color: #708dc7;" +
            "font: Gilroy;" +
            "font-size: 20px;}" +
            "*:pressed {background-color:#C5DFF1;}" +
            "*:hover {background-color: #C5DFF1;}"
        )
        botonBox.button(QDialogButtonBox.Cancel).setCursor(Qt.PointingHandCursor)
        botonBox.rejected.connect(self.reject)
        
        # Introducimos todo en un Layout
        contenedor = QVBoxLayout()
        contenedor.addWidget(texto)
        contenedor.addStretch(0)
        contenedor.addWidget(direccion)
        contenedor.addStretch(0)
        contenedor.addWidget(botonBox)
        
        # Introducimos el Layout dentro del Widget
        self.setLayout(contenedor)
        self.setFocus()
        self.exec()
        
    def keyPressEvent(self, evt):
        self.direccion.setFocus()
        self.direccion.keyPressEvent(evt)
        
    def enviar_direccion(self, direc):
        global session
        session = BaseXClient.Session('localhost', 1984, 'admin', 'admin')
        lat, lon = posicionarMapaConCalle(direc)
        cargaDBradio(lat, lon, 50, session)
        mapaD = MapaD(posicionarMapaConCalle(direc))
        ventanaP.setCentralWidget(mapaD)
        self.close()


class MapaD(QWidget):      
    def __init__(self, funcionMapa):
        super().__init__()
        
        # Definimos el fondo
        self.fondo = QLabel(self)
        pixmap = QPixmap('img/fondo-grande.jpg')
        self.fondo.setPixmap(pixmap)
        self.fondo.setScaledContents(True)
        self.fondo.setGeometry(0, 0, int(anchoP*1.5), int(alturaP*1.5))
        
        # Creamos el mapa
        mapa = QWebEngineView()
        mapa.load(QUrl.fromLocalFile(os.path.realpath("mapa.html")))
        mapa.setFixedSize(int(anchoP*3/5), alturaP)
        mapaL = QVBoxLayout()
        mapaL.addWidget(mapa)
         
        # Colocamos los filtros
        lista = QGridLayout()
        nombresF = ['Calles con puntos muertos',
                    'Calles sin entradas ni salidas',
                    'Calles sin ningún tipo de salida',
                    'Calles sin ningún tipo de entrada',
                    'Rotondas sin salidas',
                    'Rotondas sin entradas',
                    'Caminos desconectados',
                    'Áreas cerradas intersecadas por vías',
                    'Superposición de carreteras y edificios']
        nombresT = ['NoDeadlockTest',
                     'NoIsolatedWayTest',
                     'ExitWayTest',
                     'EntranceWayTest',
                     'ExitRAboutTest',
                     'EntRAboutTest',
                     'ConnectedTest',
                     'AreaNoIntTest',
                     'NoOverlapTest']
        infosF = ['Al seleccionar este filtro, buscaremos\naquellos puntos que presenten direcciones\nopuestas y no exista salida',
                  'Al seleccionar este filtro, buscaremos\naquellas vías que no se crucen con ninguna otra vía',
                  'Al seleccionar este filtro, buscaremos\naquellas vias que tengan al menos una salida',
                  'Al seleccionar este filtro, buscaremos\naquellas vias que tengan al menos una entrada',
                  'Al seleccionar este filtro, buscaremos\nrotondas cuyos puntos no que no tengan salidas',
                  'Al seleccionar este filtro, buscaremos\nrotondas que no tengan entradas',
                  'Al seleccionar este filtro, buscaremos\nque existan al menos una forma conectada a la etiqueta',
                  'Al seleccionar este filtro, buscaremos\naquellas áreas que estén intersectadas',
                  'Al seleccioanr este filtro, buscaremos\naquellas carreteras y edificios que tengan segmentos en común']
        
        self.checkboxes = []
        for i, nom in enumerate(nombresF):
            checkbox = QCheckBox(nom)
            info = infosF[i] if i < len(infosF) else ''
            checkbox.setToolTip(info)
            checkbox.setStyleSheet(
                "*{color: black;" +
                "font: Gilroy;" +
                "font-size: 20px;}" +
                "QToolTip {font-size: 18px;}"
            )
            checkbox.setCursor(Qt.PointingHandCursor)
            lista.addWidget(checkbox, i, 0)
            self.checkboxes.append(checkbox)
            
        for checkbox in self.checkboxes:
            checkbox.stateChanged.connect(self.activar_boton)
        
        #Colocamos opciones
        lista2 = QButtonGroup()
        rb1 = QRadioButton("Centrar búsqueda en posición original")
        rb1.setChecked(True)
        rb1.setStyleSheet(
                "*{color: black;" +
                "font: Gilroy;" +
                "font-size: 20px;}" +
                "QToolTip {font-size: 18px;}"
            )
        rb2 = QRadioButton("Buscar sobre posición original")
        rb2.setStyleSheet(
                "*{color: black;" +
                "font: Gilroy;" +
                "font-size: 20px;}" +
                "QToolTip {font-size: 18px;}"
            )
        rb3 = QRadioButton("Buscar en un radio de 10KM sobre posicion original")
        rb3.setStyleSheet(
                "*{color: black;" +
                "font: Gilroy;" +
                "font-size: 20px;}" +
                "QToolTip {font-size: 18px;}"
            )
        lista2.addButton(rb1, 0)
        lista2.addButton(rb2, 1)
        lista2.addButton(rb3, 2)
        
        # Contenedor Filtros
        texto = QLabel("Seleccione los errores que desea buscar:")
        texto.setWordWrap(True)
        texto.setStyleSheet(
            "*{color: white;" +
            "font: Gilroy;" +
            "font-size: 23px;" +
            "font-weight: bold;}"
        )
        
        #Contenedor Opciones
        texto2 = QLabel("Selecione la opción de búsqueda:")
        texto2.setWordWrap(True)
        texto2.setStyleSheet(
            "*{color: white;" +
            "font: Gilroy;" +
            "font-size: 23px;" +
            "font-weight: bold;}"
        )
        
        # Botones
        botonL = QHBoxLayout()
        self.b1 = BotonRedondeadoMini('EJECUTAR', 150, 70)
        self.b2 = BotonRedondeadoMini('VOLVER A MENÚ', 150, 70)
        self.b1.clicked.connect(lambda:filtros_seleccionados(self.checkboxes, nombresT))
        self.b1.clicked.connect(lambda: zona_busqueda(lista2))
        self.b1.clicked.connect(self.ver_mapaR)
        self.b2.clicked.connect(self.ver_menu)
        self.b1.setEnabled(False)
        botonL.addStretch(0)
        botonL.addWidget(self.b1)
        botonL.addWidget(self.b2)
        
        filtroL = QVBoxLayout()
        filtroL.addWidget(texto)
        filtroL.addLayout(lista)
        filtroL.addSpacerItem(QSpacerItem(0, 35, QSizePolicy.Minimum, QSizePolicy.Expanding))
        filtroL.addWidget(texto2)
        filtroL.addWidget(rb1)
        filtroL.addWidget(rb2)
        filtroL.addWidget(rb3)
        filtroL.addSpacerItem(QSpacerItem(0, 35, QSizePolicy.Minimum, QSizePolicy.Expanding))
        filtroL.addLayout(botonL)

        # Contenedor General
        mapaW = QWidget()
        mapaW.setLayout(mapaL)

        filtroW = QWidget()
        filtroW.setLayout(filtroL)

        main = QHBoxLayout()
        main.addWidget(mapaW)
        main.addWidget(filtroW)

        self.setLayout(main)
    
    def activar_boton(self):
        activar = any(checkbox.isChecked() for checkbox in self.checkboxes)
        self.b1.setEnabled(activar)
        
    def ver_mapaR(self):
        global session
        query = session.query("let $doc := doc('osm') return string(($doc//way)[1]/tag[@k='name']/@v)")
        result = query.execute()

        # Obtener el resultado como una cadena de texto
        nombre = result.strip()
        
        mapaD = MapaR(posicionarMapaConCalle(nombre))
        self.parent().setCentralWidget(mapaD)
        self.close()
        
    def ver_menu(self):
        global session
        session.close()
        menu = Menu()
        self.parent().setCentralWidget(menu)
        self.close()
        
class MapaR(QWidget):      
    def __init__(self, funcionMapa):
        super().__init__()
        
        # Definimos el fondo
        self.fondo = QLabel(self)
        pixmap = QPixmap('img/fondo-grande.jpg')
        self.fondo.setPixmap(pixmap)
        self.fondo.setScaledContents(True)
        self.fondo.setGeometry(0, 0, int(anchoP*1.5), int(alturaP*1.5))
        
        # Creamos el mapa
        mapa = QWebEngineView()
        mapa.load(QUrl.fromLocalFile(os.path.realpath("mapaR.html")))
        mapa.setFixedSize(int(anchoP*3/5), alturaP)
        mapaL = QVBoxLayout()
        mapaL.addWidget(mapa)
            
        # Contenedor Filtros
        texto = QLabel("Resultados encontrados:")
        texto.setWordWrap(True)
        texto.setStyleSheet(
            "*{color: white;" +
            "font: Gilroy;" +
            "font-size: 23px;" +
            "font-weight: bold;}"
        )
        
        texto2 = QLabel("Resultados OSM:")
        texto2.setWordWrap(True)
        texto2.setStyleSheet(
            "*{color: white;" +
            "font: Gilroy;" +
            "font-size: 23px;" +
            "font-weight: bold;}"
        )
        
        arriba = QTextBrowser(self)
        arriba.setPlainText(strErrores)
        arriba.setStyleSheet( "*{color: black;" +
            "font: Gilroy;" +
            "font-size: 16px;" +
            "font-weight: bold;}")
        arriba.setStyleSheet("background-color: lightyellow; color: balck; font-size: 16px;")
        abajo = QTextBrowser(self)
        abajo.setPlainText(strXml)
        abajo.setStyleSheet( "*{color: black;" +
            "font: Gilroy;" +
            "font-size: 16px;" +
            "font-weight: bold;}")
        abajo.setStyleSheet("background-color: lightyellow; color: balck; font-size: 16px;")
        b_export1 = BotonRedondeadoMini("Exportar Resultados", 400, 40)
        b_export2 = BotonRedondeadoMini("Exportar Resultados", 400, 40)
        nombre = "Datos"
        nombre2 = "DatosOSM"
        b_export1.clicked.connect(lambda:self.exportar(strErrores, f"{nombre}{datetime.datetime.now().strftime('%Y-%m-%d%H-%M-%S')}.pdf"))
        b_export2.clicked.connect(lambda:self.exportar(strXml, f"{nombre2}{datetime.datetime.now().strftime('%Y-%m-%d%H-%M-%S')}.pdf"))
        
        # Botones
        botonL = QHBoxLayout()
        b1 = BotonRedondeadoMini('NUEVA BÚSQUEDA', 150, 70)
        b2 = BotonRedondeadoMini('VOLVER A MENÚ', 150, 70)
        b1.clicked.connect(self.ver_mapaD)
        b2.clicked.connect(self.ver_menu)
        botonL.addStretch(0)
        botonL.addWidget(b1)
        botonL.addWidget(b2)

        filtroL = QVBoxLayout()
        filtroL.addWidget(texto)
        filtroL.addWidget(arriba)
        filtroL.addWidget(b_export1)
        filtroL.addSpacerItem(QSpacerItem(0, 5, QSizePolicy.Minimum, QSizePolicy.Expanding))
        filtroL.addWidget(texto2)
        filtroL.addWidget(abajo)
        filtroL.addWidget(b_export2)
        filtroL.addSpacerItem(QSpacerItem(0, 35, QSizePolicy.Minimum, QSizePolicy.Expanding))
        filtroL.addLayout(botonL)

        # Contenedor General
        mapaW = QWidget()
        mapaW.setLayout(mapaL)

        filtroW = QWidget()
        filtroW.setLayout(filtroL)

        main = QHBoxLayout()
        main.addWidget(mapaW)
        main.addWidget(filtroW)

        self.setLayout(main)
        
    def ver_mapaD(self):
        global session
        query = session.query("let $doc := doc('osm') return string(($doc//way)[1]/tag[@k='name']/@v)")
        result = query.execute()

        # Obtener el resultado como una cadena de texto
        nombre = result.strip()
        
        mapaD = MapaD(posicionarMapaConCalle(nombre))
        self.parent().setCentralWidget(mapaD)
        self.close()
        
    def ver_menu(self):
        global session
        session.close()
        menu = Menu()
        self.parent().setCentralWidget(menu)
        self.close()
        
    def exportar(self, datos, nombre):
        if not nombre.endswith('.pdf'):
            nombre += '.pdf'

        #Estilo PDF
        c = canvas.Canvas(nombre)
        c.setPageSize((400, 400))
        c.setFont("Helvetica", 12)
        c.drawString(50, 280, datos)
        c.save()
        popup = QMessageBox()
        popup.setWindowTitle("Exportación completa")
        popup.setText("Se ha realizado el export correctamente")
        popup.exec_()

In [ ]:
#MAIN
if __name__ == '__main__':
    app = QApplication(sys.argv)
    anchoP, alturaP, anchoV, alturaV = dimension()
    
    inicio = Inicio(anchoV,alturaV,anchoP,alturaP)
    t = AvanceBarra(inicio)
    t.start()
    
    ventanaP = QMainWindow()
    ventanaP.setCentralWidget(inicio)
    
    ventanaP.setWindowTitle("MapFlaws")
    ventanaP.setGeometry(anchoV,alturaV,anchoP,alturaP)
    ventanaP.setWindowIcon(QIcon('img/logo2.ico'))
    ventanaP.show()

    sys.exit(app.exec_())

In [1]:
session.close()

NameError: name 'session' is not defined